# 與 OpenBnB MCP 伺服器整合的 Semantic Kernel

本筆記本展示了如何使用 Semantic Kernel 與實際的 OpenBnB MCP 伺服器進行整合，透過 MCPStdioPlugin 搜尋真實的 Airbnb 住宿。對於 LLM 的存取，使用的是 Azure AI Foundry。要設定您的環境變數，您可以參考 [設定課程](/00-course-setup/README.md)。


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## 建立 MCP 插件連線

我們將使用 MCPStdioPlugin 連接到 [OpenBnB MCP 伺服器](https://github.com/openbnb-org/mcp-server-airbnb)。此伺服器透過 @openbnb/mcp-server-airbnb 套件提供 Airbnb 搜尋功能。


## 建立客戶端

在此範例中，我們將使用 Azure AI Foundry 來存取 LLM。請確保您的環境變數已正確設定。


## 環境設定

設定 Azure OpenAI 的相關配置。請確保已設定以下環境變數：
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## 理解 OpenBnB MCP 整合

此筆記本連接到 **真正的 OpenBnB MCP 伺服器**，提供實際的 Airbnb 搜尋功能。

### 運作方式：

1. **MCPStdioPlugin**：使用標準輸入/輸出與 MCP 伺服器進行通訊  
2. **真正的 NPM 套件**：透過 npx 下載並執行 `@openbnb/mcp-server-airbnb`  
3. **即時數據**：從 Airbnb 的 API 返回真實的房源數據  
4. **功能探索**：代理會自動探索 MCP 伺服器提供的可用功能  

### 可用功能：

OpenBnB MCP 伺服器通常提供以下功能：  
- **search_listings** - 根據地點和條件搜尋 Airbnb 房源  
- **get_listing_details** - 獲取特定房源的詳細資訊  
- **check_availability** - 檢查特定日期的可用性  
- **get_reviews** - 獲取房源的評論  
- **get_host_info** - 獲取房源主人的資訊  

### 先決條件：

- 您的系統上需安裝 **Node.js**  
- **網路連線** 用於下載 MCP 伺服器套件  
- **NPX** 可用（隨 Node.js 一起提供）  

### 測試連接：

您可以手動測試 MCP 伺服器，執行以下指令：  
```bash
npx -y @openbnb/mcp-server-airbnb
```  

此指令將下載並啟動 OpenBnB MCP 伺服器，Semantic Kernel 隨後會連接到該伺服器以獲取真實的 Airbnb 數據。  


## 使用 OpenBnB MCP 伺服器運行代理

現在我們將運行連接到 OpenBnB MCP 伺服器的 AI 代理，搜尋斯德哥爾摩適合 2 位成人和 1 位小孩的真實 Airbnb 住宿。您可以隨意更改 `user_inputs` 列表來修改搜尋條件。


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# 摘要
恭喜你！你已成功建立了一個能夠使用模型上下文協議（MCP）整合真實世界住宿搜尋的 AI 代理：

## 使用的技術：
- Semantic Kernel - 用於使用 Azure OpenAI 構建智能代理
- Azure AI Foundry - 提供 LLM 功能和聊天完成
- MCP（模型上下文協議）- 用於標準化工具整合
- OpenBnB MCP Server - 提供真實 Airbnb 搜尋功能
- Node.js/NPX - 用於運行外部 MCP 伺服器

## 你學到了什麼：
- MCP 整合：將 Semantic Kernel 代理連接到外部 MCP 伺服器
- 實時數據訪問：通過即時 API 搜尋真實的 Airbnb 物業
- 協議通信：使用標準輸入輸出進行代理與 MCP 伺服器之間的通信
- 功能發現：自動發現 MCP 伺服器提供的可用功能
- 流式響應：實時捕獲並記錄功能調用
- HTML 渲染：以樣式化表格和互動顯示格式化代理回應

## 下一步：
- 整合更多 MCP 伺服器（如天氣、航班、餐廳）
- 構建結合 MCP 和 A2A 協議的多代理系統
- 為自己的數據來源創建自定義 MCP 伺服器
- 實現跨會話的持久對話記憶
- 將代理部署到 Azure Functions，並進行 MCP 伺服器協調
- 添加用戶身份驗證和預訂功能



---

**免責聲明**：  
本文件使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 進行翻譯。儘管我們努力確保翻譯的準確性，但請注意，自動翻譯可能包含錯誤或不準確之處。原始文件的母語版本應被視為權威來源。對於關鍵信息，建議使用專業人工翻譯。我們對因使用此翻譯而引起的任何誤解或誤釋不承擔責任。
